In [1]:
import sys
import os
import argparse
import math
import time
from scipy.io import loadmat
import fastdyn_fic_dmf as dmf
import numpy as np
from multiprocessing import Pool

def compute_fcd(data, wsize, overlap, isubdiag, params):
    T, N = data.shape
    win_start = np.arange(0, T - params["wsize"] - 1, params["wsize"] - overlap)
    nwins = len(win_start)
    fcd = np.zeros((len(isubdiag[0]), nwins))
    for i in range(nwins):
        tmp = data[win_start[i]:win_start[i] + params["wsize"] + 1, :]
        cormat = np.corrcoef(tmp.T)
        fcd[:, i] = cormat[isubdiag[0], isubdiag[1]]
    return fcd

def grid_step(args):
    """
    Processes a single (DECAY, LR) pair with a specific seed and returns the results.
    """
    (idx_DECAY, DECAY, LR, seed, params, nb_steps, burnout, overlap, isubfcd, G_range, node_strength, post_burnout_steps) = args
    print(f"Starting grid step: DECAY={DECAY}, LR={LR}, Seed={seed}")
    OBJ_RATE = 3.44
    params['lrj'] = LR
    params['taoj'] = DECAY
    params['obj_rate'] = OBJ_RATE
    params['seed'] = seed

    # Initialize the results dictionary
    result = {
        'idx_DECAY': idx_DECAY,
        'idx_LR': None,  # Will set after enumerating LR_range
        'all_random_fic_cor': None  # Will hold the correlation data
    }

    # Iterate over G_range
    all_random_fic_cor = np.zeros((len(G_range), post_burnout_steps), dtype=np.float32)

    for idx_G, G_val in enumerate(G_range):
        params['G'] = G_val
        # Reference FIC calculation
        reference_fic = 0.75 * params['G'] * params['C'].sum(axis=0).squeeze() + 1
        # Create random vector in the same range as the reference fic
        params['J'] = np.random.rand(params['C'].shape[0]) * reference_fic.max()

        # Run the simulation
        rates, _, _, fic_t = dmf.run(params, nb_steps)

        # Extract the post-burnout part of the FIC timeseries
        # fic_t shape: (nodes, time)
        burnout_steps = int(math.ceil(burnout * 1000))
        fic_t_post_burnout = fic_t[:, burnout_steps:]

        # Compute correlation at each time step
        # For each time t, correlate fic_t_post_burnout[:, t] with node_strength
        for t in range(post_burnout_steps):
            fic_vector = fic_t_post_burnout[:, t]
            if np.std(fic_vector) == 0 or np.std(node_strength) == 0:
                cor = 0  # Handle zero variance
            else:
                cor = np.corrcoef(fic_vector, node_strength)[0, 1]
            all_random_fic_cor[idx_G, t] = cor

    # Set the results
    result['idx_LR'] = None  # Will be set by the caller based on LR_range
    result['all_random_fic_cor'] = all_random_fic_cor

    return result

def integrate_results(total_tasks, results_folder, fic_cor_timeseries_grid_shape, output_folder):
    """
    Integrates partial results from all tasks and saves the aggregated results.
    """
    print("Integrating partial results...")
    
    fic_cor_timeseries_grid = np.zeros(fic_cor_timeseries_grid_shape, dtype=np.float32)
    
    for task_idx in range(total_tasks):
        partial_file = os.path.join(results_folder, f"partial_result_{task_idx}.npy")
        if not os.path.exists(partial_file):
            print(f"Partial result file {partial_file} not found. Skipping.")
            continue
        partial_results = np.load(partial_file, allow_pickle=True)
        for partial in partial_results:
            idx_DECAY = partial['idx_DECAY']
            idx_LR = partial['idx_LR']
            all_random_fic_cor = partial['all_random_fic_cor']  # shape (len(G_range), post_burnout_steps)

            fic_cor_timeseries_grid[idx_DECAY, idx_LR] = all_random_fic_cor

    # Save integrated results
    arrays_to_save = {
        'fic_cor_timeseries_grid': fic_cor_timeseries_grid
    }

    os.makedirs(output_folder, exist_ok=True)

    for array_name, array_data in arrays_to_save.items():
        file_name = os.path.join(output_folder, f"{array_name}.npy")
        np.save(file_name, array_data)
        print(f"Saved integrated {array_name} to {file_name}")

ModuleNotFoundError: No module named '_DYN_FIC_DMF'

In [ ]:


def main():
    parser = argparse.ArgumentParser(description="Distributed FIC Correlation Simulation Script")
    parser.add_argument('--task_idx', type=int, required=True, help='Task index (0 to total_tasks-1)')
    parser.add_argument('--seed', type=int, required=True, help='Seed integer - will be used in all jobs')
    args = parser.parse_args()

    task_idx = args.task_idx
    total_tasks = 8  # Based on SLURM array=0-7

    # Prepare parameters and data
    C = loadmat('./data/DTI_fiber_consensus_HCP.mat')['connectivity'][:200, :200]
    C = 0.2 * C / np.max(C)
    params = dmf.default_params(C=C)

    triu_idx = np.triu_indices(C.shape[1], 1)
    params['N'] = C.shape[0]
    isubfcd = np.triu_indices(C.shape[1], 1)

    # Main simulation setup
    params["return_rate"] = True
    params["return_bold"] = False
    params["return_fic"] = True  # Ensure fic_t is returned
    params["with_plasticity"] = True
    params["with_decay"] = True

    G_range = np.arange(0, 8.5, 0.5)
    LR_range = np.logspace(0, 3, 100)
    DECAY_range = np.logspace(2, 6, 110)

    burnout = 7
    nb_steps = 50000
    NUM_CORES = 24
    N_RANDOMIZATIONS = 8  # Removed loop; now handled via seeds

    node_strength = C.sum(axis=0)
    # Number of time steps after the burnout period
    post_burnout_steps = nb_steps - burnout * 1000

    
    seed = args.seed

    # Create a list of argument tuples for the nested loop function
    args_list = [(
        idx_DECAY,
        DECAY,
        LR,
        seed,
        params.copy(),
        nb_steps,
        burnout,
        29,  # overlap
        isubfcd,
        G_range,
        node_strength,
        post_burnout_steps
    ) for idx_DECAY, DECAY in enumerate(DECAY_range)
      for idx_LR, LR in enumerate(LR_range)]

    # Define the folder to save partial results
    partial_results_folder = f"./Results/Figure1/FicvsStr3-44-Grid/PartialResults_seed{seed}"
    os.makedirs(partial_results_folder, exist_ok=True)

    # Define the folder to save the integrated results
    output_folder = f"./Results/Figure1/FicvsStr3-44-Grid_seed{seed}"
    os.makedirs(output_folder, exist_ok=True)

    # Process assigned (DECAY, LR) pairs using multiprocessing Pool
    with Pool(processes=NUM_CORES) as pool:
        results = pool.map(grid_step, args_list)

    # Prepare data to save
    # Each result is a dictionary: {'idx_DECAY': ..., 'idx_LR': ..., 'all_random_fic_cor': ...}
    partial_results = []
    for res in results:
        # Since each task handles one seed, set idx_LR accordingly
        idx_DECAY = res['idx_DECAY']
        # Find the corresponding idx_LR from the order of args_list
        # Assuming DECAY_range is outer loop, LR_range is inner loop
        idx_LR = res['idx_LR'] if res['idx_LR'] is not None else None
        all_random_fic_cor = res['all_random_fic_cor']
        partial_results.append({
            'idx_DECAY': idx_DECAY,
            'idx_LR': idx_LR,
            'all_random_fic_cor': all_random_fic_cor
        })

    # Save partial results outside the pool
    partial_file = os.path.join(partial_results_folder, f"partial_result_{task_idx}.npy")
    np.save(partial_file, partial_results)
    print(f"Task {task_idx}: Saved partial results to {partial_file}")

    # If this is the designated integrator task, perform integration
    # For example, task_idx=0 acts as the integrator
    if task_idx == 0:
        print("Integrator task started. Waiting for all partial results...")
        expected_files = [os.path.join(partial_results_folder, f"partial_result_{i}.npy") for i in range(total_tasks)]
        
        while True:
            existing_files = [f for f in expected_files if os.path.exists(f)]
            if len(existing_files) >= total_tasks:
                print("All partial results found. Proceeding to integrate.")
                break
            else:
                print(f"Waiting for partial results... ({len(existing_files)}/{total_tasks} files found)")
                time.sleep(60)  # Wait for 60 seconds before checking again

        # Define the shape for fic_cor_timeseries_grid
        fic_cor_timeseries_grid_shape = (
            len(DECAY_range),
            len(LR_range),
            len(G_range),
            post_burnout_steps
        )

        # Initialize the aggregated grid
        fic_cor_timeseries_grid = np.zeros(fic_cor_timeseries_grid_shape, dtype=np.float32)

        for task_idx_integrate in range(total_tasks):
            partial_file_integrate = os.path.join(partial_results_folder, f"partial_result_{task_idx_integrate}.npy")
            if not os.path.exists(partial_file_integrate):
                print(f"Partial result file {partial_file_integrate} not found. Skipping.")
                continue
            partial_results_integrate = np.load(partial_file_integrate, allow_pickle=True)
            for partial in partial_results_integrate:
                idx_DECAY = partial['idx_DECAY']
                idx_LR = partial['idx_LR']
                all_random_fic_cor = partial['all_random_fic_cor']  # shape (len(G_range), post_burnout_steps)

                # Aggregate the correlations by averaging across seeds
                # Since each task corresponds to one seed, we'll average over seeds
                fic_cor_timeseries_grid[idx_DECAY, idx_LR] += all_random_fic_cor

        # Average the aggregated correlations by the number of randomizations
        fic_cor_timeseries_grid /= N_RANDOMIZATIONS

        # Save the integrated results
        arrays_to_save = {
            'fic_cor_timeseries_grid': fic_cor_timeseries_grid
        }

        for array_name, array_data in arrays_to_save.items():
            file_name = os.path.join(output_folder, f"{array_name}.npy")
            np.save(file_name, array_data)
            print(f"Saved integrated {array_name} to {file_name}")

        print("Integration completed.")

if __name__ == "__main__":
    main()
